In [1]:
!pip install gym

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import gym
import numpy as np
import random

In [3]:
# Loading the Frozen lake environment
env = gym.make("FrozenLake-v1") 

import os
os.environ['SDL_VIDEODRIVER']='dummy'


/usr/local/lib/python3.9/dist-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/usr/local/lib/python3.9/dist-packages/gym/core.py:317: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(
/usr/local/lib/python3.9/dist-packages/gym/wrappers/step_api_compatibility.py:39: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  deprecation(


In [4]:
print("Action space: ", env.action_space)
print("Observation space: ", env.observation_space)

Action space:  Discrete(4)
Observation space:  Discrete(16)



Action Space:<br>
Indicates which direction to move the player
<ul>
<li>0 - Move Left</li>
<li>1 - Move Down</li>
<li>2 - Move Right</li>
<li>3 - Move Up</li>
</ul>

In [5]:
total_episodes = 10000
lr = 0.01
max_steps = 100
gamma = 0.99

epsilon = 1
max_epsilon = 1
min_epsilon = 0.01
decay_rate = 0.001

In [6]:
policy = np.zeros((env.observation_space.n, env.action_space.n))

In [7]:
rewards = []

for episode in range(total_episodes):
  state = env.reset()
  step = 0
  done = False
  total_reward = 0

  for step in range(max_steps):
    if random.uniform(0,1) > epsilon:
      action = np.argmax(policy[state,:])                         # Exploitation
    else:
      action = env.action_space.sample()                          # Exploration
    
    new_state, reward, done, info = env.step(action)
    max_new_state = np.max(policy[new_state,:])
    policy[state,action] += lr * (reward + gamma*max_new_state-policy[state,action])

    total_reward += reward
    state = new_state

    if done:
      break

  epsilon = min_epsilon + (max_epsilon - min_epsilon)*np.exp(-decay_rate*episode)
  rewards.append(total_reward)

print("Score: ",str(sum(rewards)/total_episodes))

Score:  0.1028


<h3>Q1. Create a Policy Iteration function with the following parameters</h3>
<ol>
<li>policy: 2D array of a size n(S) x n(A), each cell represents a probability of taking action a in state s.
<li>environment: Initialized OpenAI gym environment object
<li>discount_factor: MDP discount factor
<li>theta:  A  threshold  of  a  value  function  change.  Once  the  update  to  value function is below this number
<li>max_iterations: Maximum number of iterations
</ol>

In [8]:
import numpy as np
import gym
from gym import wrappers
from gym.envs.registration import register

In [9]:
def policy_evalaution(env, policy, discount_factor):
    # Initializing the value_function to zero
    v = np.zeros(env.observation_space.n)

    # Initializing some variables/terms
    # theta = 1e-10
    theta = 1e-5
    i=0
    # Update the value function until the change is below the theta value
    while True:
        # print("new iteration\n"+str(i++))
        delta = 0
        # Store the value function before updating
        prev_v = np.copy(v)
        # Iterate through the state space
        for s in range(env.observation_space.n):
            # Choose action for state s based on policy
            policy_a = policy[s]
            # Update the value function
            v[s] = sum([p * (r + discount_factor * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
            # print(v[s])
            delta = max(delta,(np.abs(prev_v[s] - v[s])))
            # print(np.fabs(prev_v[s] - v[s]))
            # print("Delta: " + str(delta))
        # print(v)
        # Condition to break the loop
        delta = np.sum((np.fabs(prev_v - v)))
        if delta < theta:
            # print("Converged\n\n")
            # value converged
            break

    # returning the value function
    return v

In [10]:
def policy_evalaution(env, policy, discount_factor):
    # Initializing the value_function to zero
    v = np.zeros(env.observation_space.n)

    # Initializing some variables/terms
    theta = 1e-10
    delta = 0
    i=0
    # Update the value function until the change is below the theta value
    while True:
        # Store the value function before updating
        prev_v = np.copy(v)
        # Iterate through the state space
        for s in range(env.observation_space.n):
            # Choose action for state s based on policy
            policy_a = policy[s]
            # Update the value function
            v[s] = sum([p * (r + discount_factor * prev_v[s_]) for p, s_, r, _ in env.P[s][policy_a]])
        if i < 100: print(v)
        i +=1
        # Condition to break the loop
        delta = np.sum((np.fabs(prev_v - v)))
        if delta <= theta:
            # value converged
            break

    # returning the value function
    return v

In [11]:
def policy_improvement(v, discount_factor):
    # Initialize a policy for the state space
    policy = np.zeros(env.observation_space.n)

    # Iterate through the state space
    for s in range(env.observation_space.n):
        # Initialize the Estimated action value
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            # Update the Initialize the Estimated action value for each action based on the value fucntion
            q_sa[a] = sum([prb * (rew + discount_factor * v[st]) for prb, st, rew, _ in  env.P[s][a]])
        # Update the policy based on the Estimated action value
        policy[s] = np.argmax(q_sa)

    # Return the new policy
    return policy

In [12]:
def policy_iteration(env, discount_factor):
    # Either choose a random initial policy or zero policy
    # policy = np.random.choice(env.action_space.n, size=(env.observation_space.n))  # initialize a random policy
    policy = np.zeros(env.observation_space.n, dtype=int)

    # Set the max no. of iterations
    max_iterations = 1000
    for i in range(max_iterations):
        # Find the value function
        value_function = policy_evalaution(env, policy, discount_factor)
        # Obtain new policy based on the found value function
        new_policy = policy_improvement(value_function, discount_factor)
        # Ckeck whether the obtained policy is same as existing policy
        if (np.all(policy == new_policy)):
            print ('Policy-Iteration converged at step %d.' %(i+1))
            break
        # Update the policy
        policy = new_policy
    
    # Return the optimal policy
    return policy

In [13]:
def run_episodes(env, policy, discount_factor, render = False):
    """ Runs an episode and return the total reward """
    # Set number of episodes
    num_episode = 1000
    episode_reward = []

    wins = 0
    total_reward = 0
    total_action = 0

    # Iterate through the episodes
    for i in range(num_episode):
      # Reset the Environment
      obs = env.reset()
      # Initialize the variables
      total_reward = 0
      step_idx = 0

      while True:
          if render:
              env.render()
          # Take action based on policy
          obs, reward, done , _ = env.step(int(policy[obs]))
          # Update the cumulative reward
          # total_reward += (discount_factor ** step_idx * reward)
          total_reward += reward
          step_idx += 1
          # Check if reached Goal State
          if done:
              wins += 1
              break
      # Add the episode reward to a List
      episode_reward.append(total_reward)

      # if i%10 ==0:
      #   print(i)

    # Return list of rewards for 1000 episodes
    return episode_reward, wins

In [14]:
discount_factor = 0.99

# Load the Environment
env = gym.make("FrozenLake-v1", is_slippery=True) 

# Find the Optimal Policy
optimal_policy = policy_iteration(env, discount_factor)

action_mapping = {
      3: '\u2191',  # up
      2: '\u2192',  # right
      1: '\u2193',  # down
      0: '\u2190'   # left
}
print(' '.join([action_mapping[action] for action in optimal_policy]))

# Apply the Optimal Policy
scores, wins = run_episodes(env, optimal_policy, discount_factor, render=False)
# Find the Average reward score
print("Total number of Episodes = 1000")
print('Average reward = ', np.mean(scores))
print("Total number of wins: ",wins)

[0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.33333333 0.        ]
[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.11       0.
 0.         0.         0.44333333 0.        ]
[0.         0.         0.         0.         0.         0.
 0.0363     0.         0.         0.         0.1463     0.
 0.         0.         0.47963333 0.        ]
[0.         0.         0.011979   0.         0.         0.
 0.048279   0.         0.         0.         0.170258   0.
 0.         0.         0.49161233 0.        ]
[0.         0.         0.01988514 0.00395307 0.         0.
 0.06013821 0.         0.         0.         0.17816414 0.
 0.         0.         0.4955654  0.        ]
[0.         0.         0.02640771 0.00786661 0.         0.
 0.06535626 0.         0.         0.         0.18338219 0.
 0.         

<h3>Q2. Create a Value Iteration function with the following parameters</h3>
<ol>
<li>environment: Initialized OpenAI gym environment object
<li>discount_factor: MDP discount factor
<li>theta:  A  threshold  of  a  value  function  change.  Once  the  update  to  value function is below this number
<li>max_iterations: Maximum number of iterations
</ol>

In [15]:
def extract_policy(v, discount_factor = 0.99):
    # Initialize a policy for the state space
    policy = np.zeros(env.observation_space.n)

    # Iterate through the state space
    for s in range(env.observation_space.n):
        q_sa = np.zeros(env.action_space.n)
        for a in range(env.action_space.n):
            for next_sr in env.P[s][a]:
                # next_sr is a tuple of (probability, next state, reward, done)
                p, s_, r, _ = next_sr
                q_sa[a] += (p * (r + discount_factor * v[s_]))
        # Update the policy with best possible action value for the state s
        policy[s] = np.argmax(q_sa)
    
    # Return the (new) Updated Policy
    return policy

In [16]:
def value_iteration(env, discount_factor = 0.99):
    # Initialize the value-function
    v = np.zeros(env.observation_space.n)
    # Set variables
    max_iterations = 10000
    theta = 1e-20
    for i in range(max_iterations):
        # Store the value function before updating
        prev_v = np.copy(v)
        # Iterate through the State space
        for s in range(env.observation_space.n):
            # Find the Estimated Action value of all Actions for state s
            q_sa = [sum([p*(r + discount_factor * prev_v[s_]) for p, s_, r, _ in env.P[s][a]]) for a in range(env.action_space.n)]
            # Update the Value Function
            v[s] = max(q_sa)

        # Condition to break the loop
        if (np.sum(np.fabs(prev_v - v)) <= theta):
            print ('Value-iteration converged at iteration# %d.' %(i+1))
            break
    # Return the value Function
    return v

In [17]:
def run_episodes(env, policy, discount_factor, render = False):
    """ Runs an episode and return the total reward """
    # Set number of episodes
    num_episode = 1000
    episode_reward = []
    wins = 0

    # Iterate through the episodes
    for i in range(num_episode):
      # Reset the Environment
      obs = env.reset()
      # Initialize the variables
      total_reward = 0
      step_idx = 0
      while True:
          if render:
              env.render()
          # Take action based on policy
          obs, reward, done , _ = env.step(int(policy[obs]))
          # Update the cumulative reward
          # total_reward += (discount_factor ** step_idx * reward)
          total_reward += reward
          step_idx += 1
          # Check if reached Goal State
          if done:
              wins += 1
              break
      # Add the episode reward to a List
      episode_reward.append(total_reward)

    # Return list of rewards for 1000 episodes
    return episode_reward, wins

In [18]:
discount_factor = 0.99

# Load the Environment
env_name  = 'FrozenLake-v1'
env = gym.make(env_name, is_slippery=True)

# Find the Optimal Value Functioin
optimal_vf = value_iteration(env, discount_factor);
# Obtain Policy based on this value function
policy = extract_policy(optimal_vf, discount_factor)

print(' '.join([action_mapping[action] for action in policy]))

# Apply this policy and Evaluate
value_scores, wins = run_episodes(env, policy, discount_factor, render=False)
print("Using Value Iteration Function")
print("Total number of Episodes = 1000")
print('Average reward = ', np.mean(value_scores))
print("Total number of wins: ",wins)

Value-iteration converged at iteration# 996.
← ↑ ↑ ↑ ← ← ← ← ↑ ↓ ← ← ← → ↓ ←
Using Value Iteration Function
Total number of Episodes = 1000
Average reward =  0.726
Total number of wins:  1000


<h3>Q3. Compare  the number of  wins, average  return  after  1000  episodes and  comment  on which method performed better.</h3>

We can see that the average rewards for Value Iteration Function is slightly better than Policy Iteration Function.<br>
In this example Policy Iteration function has coverged faster<br>
Both methods are have similar average score over 1000 episodes